In [ ]:
#Import Spark SQL and Spark ML Libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

In [ ]:
#Load Source Data
csv = spark.read.csv('PATH', inferSchema=True, header=True)
csv.show()

In [ ]:
#Prepare the Data
data = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("Late")))
data.show()

In [ ]:
#Split the Data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print "Training Rows:", train_rows, " Testing Rows:", test_rows

In [ ]:
#Prepare the Training Data
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(train).select(col("features"), col("Late").alias("label"))
training.show()

In [ ]:
#Train a Classification Model
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
model = lr.fit(training)
print "Model trained!"

In [ ]:
#Train a Classification Model
testing = assembler.transform(test).select(col("features"), col("Late").alias("trueLabel"))
testing.show()

In [ ]:
#Test the Model
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")
predicted.show(100)